Download dataset

In [ ]:
!mkdir data

In [ ]:
import os
import glob
import csv
import tarfile
import urllib.request
import imageio
import numpy as np

tarball_url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
dirnames = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
data_root = 'data'
filename = tarball_url.split('/')[-1]
filepath = os.path.join(data_root, filename)

filepath, _ = urllib.request.urlretrieve(tarball_url, filepath)
tarfile.open(filepath, "r:gz").extractall(data_root)

Create .csv files containing image file names to be used inside your dataset class since it would be expensive to load ALL training images into memory so you may want to load only the images for each mini-batch.

TO DO: Create your own Dataset class definition and use DataLoader with it. Inside your dataset class, __getitem__() will use the stored file names with given input index in .csv file to load that image into memory.

In [ ]:
data_dir = os.path.join(data_root, "flower")

if not os.path.exists(os.path.join(data_dir, "train")):
    os.makedirs(os.path.join(data_dir, "train"))
if not os.path.exists(os.path.join(data_dir, "test")):
    os.makedirs(os.path.join(data_dir, "test"))

print("[!] Converting images...")
# manage data with csv files
train_f = open(os.path.join(data_dir, "train.csv"), "w")
test_f = open(os.path.join(data_dir, "test.csv"), "w")
train_writer, test_writer = csv.writer(train_f), csv.writer(test_f)

for dirname in dirnames:
    paths = glob.glob(os.path.join(
        data_root, "flower_photos", dirname, "*.jpg"))

    num_test = int(len(paths) * 0.2)

    if not os.path.exists(os.path.join(data_dir, "train", dirname)):
        os.makedirs(os.path.join(data_dir, "train", dirname))
    if not os.path.exists(os.path.join(data_dir, "test", dirname)):
        os.makedirs(os.path.join(data_dir, "test", dirname))

    # prepare a training data
    for path in paths[:-num_test]:
        new_path = os.path.join(data_dir, "train",
                                dirname, path.split("/")[-1])

        im = imageio.imread(path)
        imageio.imsave(new_path, im)
        train_writer.writerow([new_path, dirname])

    # prepare a test data
    for path in paths[-num_test:]:
        new_path = os.path.join(data_dir, "test",
                                dirname, path.split("/")[-1])

        im = imageio.imread(path)
        imageio.imsave(new_path, im)
        test_writer.writerow([new_path, dirname])

train_f.close()
test_f.close()

[!] Converting images...


In [ ]:
import torch.utils.data as data
from PIL import Image

def default_loader(path):
    return Image.open(path).convert('RGB')

class myImageFloder(data.Dataset):
    def __init__(self, root, label, transform = None, target_transform=None, loader=default_loader):

        fn = np.loadtxt(label, delimiter=',', dtype='str')
        '''edited'''
        imgs = []

        testlabel = fn[:, 1]
        testimg = fn[:, 0]
        count = 0
        for name in testimg:
            imgs.append((name,testlabel[count]))
            count=count+1

        self.root = root
        self.imgs = fn
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        filename, label = self.imgs[index]
        img = self.loader(filename)
        if self.transform is not None:
            img = self.transform(img)
        return img, label, filename  # todo: for testing visualization, it needs filename

    def __len__(self):
        return len(self.imgs)

    def getName(self):
        return self.classes

In [ ]:
import torchvision.transforms as transforms
import torch.utils

mytransform = transforms.Compose([

    transforms.Resize((128, 128)),
    transforms.ToTensor(),            # mmb
    ]
)

data_set = myImageFloder(
    root="data/flower/train",
    label="data/flower/train.csv",
    transform=mytransform,

)
data_set_test = myImageFloder(
    root="data/flower/test",
    label="data/flower/test.csv",
    transform=mytransform,

)
imgLoader = torch.utils.data.DataLoader(
    data_set,
    batch_size=500, shuffle=True, num_workers=0
)
imgLoader_test = torch.utils.data.DataLoader(
    data_set,
    batch_size=500, shuffle=True, num_workers=0
)

['data/flower/train/daisy/5054771689_00dd40b971_n.jpg'
 'data/flower/train/daisy/2713919471_301fcc941f.jpg'
 'data/flower/train/daisy/5809489674_5659b3ae5d_n.jpg' ...
 'data/flower/train/tulips/13531007054_c88deaf302_n.jpg'
 'data/flower/train/tulips/14025589299_eac64c51af_m.jpg'
 'data/flower/train/tulips/130685040_3c2fcec63e_n.jpg']
['data/flower/test/daisy/506018088_4f7a15a7c5_n.jpg'
 'data/flower/test/daisy/294451721_5106537b34.jpg'
 'data/flower/test/daisy/6776075110_1ea7a09dd4_n.jpg'
 'data/flower/test/daisy/705422469_ffa28c566d.jpg'
 'data/flower/test/daisy/4757448834_a29a9538c9_n.jpg'
 'data/flower/test/daisy/6910811638_aa6f17df23.jpg'
 'data/flower/test/daisy/8008258043_5457dd254b_n.jpg'
 'data/flower/test/daisy/162362896_99c7d851c8_n.jpg'
 'data/flower/test/daisy/10437754174_22ec990b77_m.jpg'
 'data/flower/test/daisy/3483303007_42e3f90da7.jpg'
 'data/flower/test/daisy/8694909523_3ca25d449d_n.jpg'
 'data/flower/test/daisy/3713290261_8a66de23ab.jpg'
 'data/flower/test/daisy/235

TO DO: Create your own model class definition and train your model with the training set


TO DO: Evalute your trained model with the test set.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Flower(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 8, 3)
        self.fc1 = nn.Linear(7200, 500)
        self.fc2 = nn.Linear(500, 124)
        self.fc3 = nn.Linear(124, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
from sklearn.preprocessing import binarize
epochs = 100
labelDict = {
  "daisy" : 1,
  "dandelion" : 2,
  "roses" : 3,
  "sunflowers" : 4,
  "tulips" : 5
}

model = Flower()
model.eval()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(epochs):
  hit = 0
  tot = 0
  batch_num = 0
  for i, data in enumerate(imgLoader, 0):
    batch_num += 1
    inputs, labels, _ = data
    lables_binary = []
    for i in enumerate(labels):
      if labelDict[i[1]] == 1:
        binary = [1,0,0,0,0]
      elif labelDict[i[1]] == 2:
        binary = [0,1,0,0,0]
      elif labelDict[i[1]] == 3:
        binary = [0,0,1,0,0]
      elif labelDict[i[1]] == 4:
        binary = [0,0,0,1,0]
      elif labelDict[i[1]] == 5:
        binary = [0,0,0,0,1]
      lables_binary.append(binary)

    labels = np.array(lables_binary)
    labels = torch.from_numpy(labels).type(torch.FloatTensor)

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    print("Training loss : {}".format(loss))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    output = outputs.detach().cpu().numpy()
    output_n = (output == output.max(axis=1)[:,None]).astype(int)
    # output_n = binarize(output.reshape(-1,1), threshold=0.5)
    # output_n = output_n.reshape((int(output_n.shape[0]/5), 5))
    for i in range(output_n.shape[0]):
      if np.array_equal(output_n[i], lables_binary[i]):
        hit += 1
        tot += 1
      else:
        tot += 1
  print("===========================================================")
  print("Training Acc on Epoch {}".format(epoch) + ": {}".format(hit/tot))
  print("===========================================================")

  hit = 0
  tot = 0
  with torch.no_grad():
    for i, data in enumerate(imgLoader_test, 0):
      batch_num += 1
      inputs, labels, _ = data
      lables_binary = []
      for i in enumerate(labels):
        if labelDict[i[1]] == 1:
          binary = [1,0,0,0,0]
        elif labelDict[i[1]] == 2:
          binary = [0,1,0,0,0]
        elif labelDict[i[1]] == 3:
          binary = [0,0,1,0,0]
        elif labelDict[i[1]] == 4:
          binary = [0,0,0,1,0]
        elif labelDict[i[1]] == 5:
          binary = [0,0,0,0,1]
        lables_binary.append(binary)

      labels = np.array(lables_binary)
      labels = torch.from_numpy(labels).type(torch.FloatTensor)

      outputs = model(inputs)

      output = outputs.detach().cpu().numpy()
      output_n = (output == output.max(axis=1)[:,None]).astype(int)
      for i in range(output_n.shape[0]):
        if np.array_equal(output_n[i], lables_binary[i]):
          hit += 1
          tot += 1
        else:
          tot += 1
    print("===========================================================")
    print("Test Acc on Epoch {}".format(epoch) + ": {}".format(hit/tot))
    print("===========================================================")



Training loss : 1.6120363473892212
Training loss : 1.60155189037323
Training loss : 1.6051491498947144
Training loss : 1.5935951471328735
Training loss : 1.5743557214736938
Training loss : 1.5875228643417358
Training Acc on Epoch 0: 0.23851650221163662
Test Acc on Epoch 0: 0.290575025518884
Training loss : 1.531959056854248
Training loss : 1.523092269897461
Training loss : 1.5164031982421875
Training loss : 1.4904650449752808
Training loss : 1.4781701564788818
Training loss : 1.4524182081222534
Training Acc on Epoch 1: 0.3337870023817625
Test Acc on Epoch 1: 0.3763184756719973
Training loss : 1.447386384010315
Training loss : 1.4325408935546875
Training loss : 1.3951083421707153
Training loss : 1.4560606479644775
Training loss : 1.3478319644927979
Training loss : 1.3745853900909424
Training Acc on Epoch 2: 0.4011568560734944
Test Acc on Epoch 2: 0.4521946240217761
Training loss : 1.3305162191390991
Training loss : 1.3538562059402466
Training loss : 1.3486459255218506
Training loss : 1.

Test Acc on Epoch 99: 0.9996597482136781
